**Load environmental variables**: See "Getting setup" in the first modele.  `dotenv` [docs](https://pypi.org/project/python-dotenv/) will look for `.env`. If it finds it, it will load environmental variables from there, overriding any variables in the current shell. If it is not found, variables currently in the shell are used.

In [ ]:
import warnings, os

warnings.filterwarnings(
    "ignore",
    message="LangSmith now uses UUID v7", 
    category=UserWarning,
)

In [ ]:
from dotenv import load_dotenv

load_dotenv()

# automatically reload all modules before executing new code. The captures changes in local packages.
%load_ext autoreload
%autoreload 2

## Create Agent  - Prebuilt

<img src="./assets/agent_header.png" width="800" style="display:block; margin-left:0;">

In this course, you're going to build a [Deep Agent](https://blog.langchain.com/deep-agents/). We'll build this on top of LangChains's 'pre-built' agent abstraction, which simplifies the code significantly. In this lesson, you'll learn about the pre-built ReAct agent. Here's what you will learn:
- What is a ReAct Agent
- The capabilities of our implementation and where to find out more.
    - Build an agent with tools
    - The graph, state and messages
    - Access and modify state with tools
    - <span style="font-size:0.8em;">🪝</span> hooks! and structured responses


### What is a ReAct agent 


<img src="./assets/agent.png"
     style="float:left; max-width:300px; height:auto; margin:0 1rem 0.5rem 0;">
<div style="max-width: 1250px;">
    
You will be using LangChain's open-source `create_agent` ([see here](https://docs.langchain.com/oss/python/langchain/agents)) abstraction. A **ReAct agent** is an AI agent that uses the "Reasoning and Acting" (ReAct) framework to combine chain-of-thought (CoT) reasoning with external tool use. It was made popular by the paper [ReAct: Synergizing Reasoning and Acting in Language Models](https://arxiv.org/abs/2210.03629). 

This agent consists of three components: a large language model (LLM), a set of tools it can use, and a prompt that provides instructions.

The LLM operates in a loop. In each iteration, it examines its context, which includes a list of available tools; It decides if it needs to call a tool. It selects a tool to invoke, forms the tool call. This is sent to a tool node for execution. The tool node executes the tool(s), and sends the results (observations) back to the LLM. The LLM receives the observations(s) and uses that observation to inform the next action. The loop continues until a stopping condition is met — typically when the agent decides it no longer needs to call more tools.
</div>

<div style="clear:both;"></div>


### Key capabilities of `create_agent`

Here are some of the features that come with the `create_agent` abstraction. We won't make use of them all in this course, but it does motivate the use of `create_agent` to build our course.:

- [Memory integration](../how-tos/memory/add-memory.md): Native support for _short-term_ (session-based) and _long-term_ (persistent across sessions) memory, enabling stateful behaviors in chatbots and assistants.
- [Human-in-the-loop control](../concepts/human_in_the_loop.md): Execution can pause _indefinitely_ to await human feedback—unlike websocket-based solutions limited to real-time interaction. This enables asynchronous approval, correction, or intervention at any point in the workflow.
- [Streaming support](../how-tos/streaming.md): Real-time streaming of agent state, model tokens, tool outputs, or combined streams.
- [Deployment tooling](../tutorials/langgraph-platform/local-server.md): Includes infrastructure-free deployment tools. [**LangGraph Platform**](https://langchain-ai.github.io/langgraph/concepts/langgraph_platform/) supports testing, debugging, and deployment.
  - [Studio](https://langchain-ai.github.io/langgraph/concepts/langgraph_studio/): A visual IDE for inspecting and debugging workflows.
  - [LangSmith](https://smith.langchain.com/): A tracing and evaluation tool.
  - Supports multiple [deployment options](https://langchain-ai.github.io/langgraph/concepts/deployment_options.md) for production.

  

<div style="background-color: #fff3b0; padding: 10px; border-radius: 4px;">
<b>Note:</b>  
The <code>create_react_agent</code> was moved from the LangGraph library to the LangChain library and renamed to <code>create_agent</code> in the 1.0 code release post-filming. The video and notebooks in this lesson have been updated to reflect this; however, in the remaining lessons, only the notebooks are updated.

In [ ]:
from typing import Annotated, List, Literal, Union

from langchain_core.messages import ToolMessage
from langchain_core.tools import InjectedToolCallId, tool
from langgraph.prebuilt import InjectedState
from langgraph.types import Command
from IPython.display import Image, display, JSON
from langchain.chat_models import init_chat_model
from langchain.agents import create_agent 
from utils import format_messages
from langchain.agents import AgentState


#### Build an agent with tools

Let's start by creating an agent with a simple calculator tool to get started. Once you see how things are put together, we'll go over more details.


###  The graph, state and messages
You'll run the agent in a moment, but let's dig into the graph a little bit. 
 If you would like to try building a simple version of this yourself, you can check out [Foundation: Introduction to LangGraph, Module 1, Lesson 6, Agent](https://academy.langchain.com/courses/take/intro-to-langgraph/lessons/58239232-lesson-6-agent).

**Defining the Agent**: When you define an agent as you did above, you provide: the model, one or more tools, a "system" prompt, and state schema which defaults to <a href="https://github.com/langchain-ai/langgraph/blob/e365b2b8bd695e03d758b19ff109152b2e342a87/libs/prebuilt/langgraph/prebuilt/chat_agent_executor.py#L62-L69">
  <code style="color:#0366d6;">AgentState</code>
</a> which is primarily a list of messages. ([Call details here.](https://langchain-ai.github.io/langgraph/reference/agents/#langgraph.prebuilt.chat_agent_executor.create_react_agent))
Under the hood, this is defining and compiling the LangGraph graph shown above. An important detail is that the tools node is another pre-built item, a `ToolNode`, described [here](https://github.com/langchain-ai/langgraph/blob/e365b2b8bd695e03d758b19ff109152b2e342a87/libs/prebuilt/langgraph/prebuilt/tool_node.py#L239-L293). A tool node will run all the tools identified in the message from the LLM and return the results.

**Invoking the Agent:** 
Let's call the agent and see what we get!

**Invoking the Agent:** 
A sequence diagram is a great way to look at what happens when the model is invoked.

<div style="display:none">
    the mermaid code is saved here for future
```mermaid
sequenceDiagram
    participant U as User
    participant A as LLM
    participant T as Tools
    Note over A: System message<br/>"You are a helpful assistant..."
    U->>A: Initial input,<br/>"What is 3.1 * 4.2?"
    loop while tool_calls present
        A->>T: AIMessage(id="call_123", tool_calls=[...])
        T-->>A: ToolMessage(tool_call_id="call_123", content="xx")
    end
    A->>U: Return final state
```
</div> 

<img src="./assets/agent_sequence_diagram.png"
     style="float:left; max-width:500px; height:auto; margin:0 1rem 0.5rem 0;">
<div style="max-width: 1100px;">
In our example, the user input is "What is 3.1 * 4.2?". This, combined with the system prompt and tool descriptions, is sent to the LLM.  
<br/>
<p style="margin-bottom:0; margin-top:5px;">The LLM decides that the calculator tool should be called. <br/> It adds an `AIMessage` to `messages`:</p>
<pre style="font-size:0.85em; margin-top:5px; margin-bottom:0;">
<code class="language-python">AIMessage(
    content="",
    tool_calls=[{"id": "call_123",
                 "name": "calculator",
                 "args": {"a": 3.1, "b": 4.2, "operation": "multiply"}}])</code></pre>   
</code></pre>
<br/>
<p style="margin-bottom:0;">The tool node receives the AIMessage and processes all the tool calls. It tracks the tool_call_ids. It responds with a ToolMessage in `messages`: </p>
<pre style="font-size:0.85em; margin-top:5px; margin-bottom:10px;">
<code class="language-python">ToolMessage(
    content="13.02",         # The result of the tool execution.
    tool_call_id="call_123")  # Matches the id from the AIMessage.tool_calls
</code></pre>
The LLM examines the response in `messages`, decides it is done, and forms an `AIMessage` to the user.

</div>

<div style="clear:both;"></div>

Let's look at a prior [trace in LangSmith](https://smith.langchain.com/public/3d2062e3-8713-4510-a797-801abe44d1f7/r). Here are some things to notice:
- In the metadata of the call to the LLM, you will see the 'calculator' tool description.
- The response from the model is a tool call with the arguments
- In the final call to the LLM, notice the matching tool_call_id's provided by the tool node.

#### Try your own
Take a moment and try this on your own. Run a query, check your own trace in LangSmith and see if it matches your expectation. Try expanding the calculator - maybe add a square function.

### Access and modify state within tools
#### State
One of the nice features of LangGraph is state. The graph has a typed data structure that is available to each node for the duration of the graph and can be persisted in long-term storage. You can use this to store information to share between nodes, to debug the graph, and to reset a long-running graph to an earlier time.

When you define state for a graph, you define the data types and a 'reducer' function. The reducer describes how information is added to that element. This is especially useful when a task is mapped to multiple nodes, which are executed in parallel and update state simultaneously.

In this example, the default `AgentState` was used. This is defined in [langgraph.prebuilt.chat_agent_executor](https://github.com/langchain-ai/langgraph/blob/e365b2b8bd695e03d758b19ff109152b2e342a87/libs/prebuilt/langgraph/prebuilt/chat_agent_executor.py).   

```python
    class AgentState(TypedDict):
        """The state of the agent."""
        messages: Annotated[Sequence[BaseMessage], add_messages]
        remaining_steps: NotRequired[RemainingSteps]
```
        
- `messages` are a list of `BaseMessage`, defined in [langchain_core](https://github.com/langchain-ai/langchain/blob/088095b663993b1e53cf616e1ca487d1739b0d71/libs/core/langchain_core/messages/base.py), which contains the messages to and from the LLM.
    - typing.Annotated allows you to attach arbitrary metadata to a type hint. Syntax: Annotated[Type, metadata1, metadata2, ...] 
- The `add_messages` reducer will append new messages to the end of the message list.  
- `remaining_steps` tracks the steps in a graph. You will see this initialized as the `recursion_limit`, but is tracked by the graph and not visibile to the user.  
Let's take a look at this quickly.

#### Custom State
Let's extend our calculator to keep a list of all of the operations that have been performed. This will require adding a list to state, and a reducer function to add the state to the list. This will safely handle the case where the list or operation is empty.

#### Accessing State 
Now, we can extend our calculator to include the update. This highlights an issue! Now state is an argument to our calculator tool. 
<img src="./assets/state_arg_diagram.png" width="800" style="display:block; margin-left:0;">
In the diagram, it's clear that, while the LLM is tasked with generating the tool call, it cannot form the `state` argument as it does not have that in its context!  
The solution is to **inject the state** after the LLM.
<img src="./assets/inject_state_diagram.png" width="1000" style="display:block; margin-left:0;"> 

<div style="margin:0; padding:0">
  <p style="margin:0;">This is accomplished with the <code>InjectedState</code> annotation as shown below.</p>
  <pre style="font-size:0.90em; margin:0; padding-top:0;">
<code class="language-python">    @tool
    def calculator_wstate(
        operation: Literal["add","subtract","multiply","divide"],
        a: Union[int, float],
        b: Union[int, float],
        <span style="background:#fff3a3; padding:0 2px;">state: Annotated[CalcState, InjectedState],</span>  # ← not sent to LLM
        <span style="background:#fff3a3; padding:0 2px;">tool_call_id: Annotated[str, InjectedToolCallId],</span>  # ← not sent to LLM
    ) -> Union[int, float]:
</code></pre>
<p style="margin:0; padding-top:4px;">
    This strips <code>state</code> from the description provided to the LLM, and injects it when calling the tool in <code>ToolNode</code>. the <code>tool_call_id</code> is also included. This is explained in the next section.
  </p>
</div>

#### Updating State
You may recall that tools typically return their observations to the LLM in a `ToolMessage` that is included in `messages` field in state. To update additional members of state, we would like to extend this update.  This is done using `Command` as in the return below. 
```python
    return Command(
        update={
            "ops": ops,
            "messages": [
                ToolMessage(f"{result}", tool_call_id=tool_call_id)
            ]})
```
Note that to create a `ToolMessage` we needed the `tool_call_id`.

Let's try one more example. Notice the dual tool call in this example. The tool node will execute these in parallel.